In [1]:
import requests
import time # Nos permite hacer pausas entre las verificaciones del estado de la solicitud para no sobrecargar la API
import sys
import json
from Bio import SeqIO
from Bio.Seq import Seq
from Bio.SeqRecord import SeqRecord
from io import StringIO

In [2]:
def lectura_secuencias_fasta (ruta_archivo):
    secuencias_archivo = list (SeqIO.parse (ruta_archivo, "fasta"))
    if len (secuencias_archivo) <2:
        print (" Se necesitan al menos dos secuencias para poder realizar el alineamiento")

    archivo_alineamiento = StringIO()
    SeqIO.write (secuencias_archivo, archivo_alineamiento, "fasta")
    return archivo_alineamiento.getvalue()
        
    

In [3]:
secuencias_alineamiento = lectura_secuencias_fasta ("data_TP53/var_patogenicas_exon4_tp53.fasta")

## Realizar el alineamiento múltiple de las variantes de un exón

In [10]:
def alineamiento_multiple (variantes_fasta, correo):
    rest_api_clustal = "https://www.ebi.ac.uk/Tools/services/rest/clustalo/run/"
    parametros_solicitud = {"email": correo, "sequence": variantes_fasta}  # Datos que vamos a enviar a la página web para su procesamiento
    solicitud = requests.post (rest_api_clustal, data = parametros_solicitud) #Guardamos en una variable el estado del la solicitud y el ID del trabajo
    
    if solicitud.status_code == 200:
        id_alineamiento = solicitud.text   #Guardamos el ID del trabajo en una variable y la imprimimos por pantalla
        print (f"El ID del alineamiento es: {id_alineamiento}")

    estado_trabajo = f"https://www.ebi.ac.uk/Tools/services/rest/clustalo/status/{id_alineamiento}"
    resultado_alineamiento_url = f"https://www.ebi.ac.uk/Tools/services/rest/clustalo/result/{id_alineamiento}/aln-clustal_num"  # Obtenemos el resultado del alineamiento en formato Clustal

    estado_respuesta = requests.get (estado_trabajo)
    estado_alineamiento = estado_respuesta.text
    
    while estado_alineamiento != "FINISHED":
        estado_respuesta = requests.get (estado_trabajo)
        estado_alineamiento = estado_respuesta.text
        if estado_alineamiento == "FINISHED":
            break
        elif estado_alineamiento == "RUNNING":
            print ("El alineamiento se está realizando ahora mismo ...")
        elif estado_alineamiento == "QUEUED":
            print ("El alineamiento está en cola ...")
        else:
            print ("El alineamiento ha fallado")
            break

    if estado_alineamiento == "FINISHED":
        resultado_alineamiento = requests.get(resultado_alineamiento_url)
        if not solicitud.ok:
            solicitud.raise_for_status()
            sys.exit()
        alineamiento = resultado_alineamiento.text
        print (alineamiento)

        return alineamiento 
        
            
    

In [17]:
alineamiento_var_patogenicas_exon4_tp53 = alineamiento_multiple (secuencias_alineamiento, "aca1033@alu.ubu.es")

El ID del alineamiento es: clustalo-R20240620-232932-0914-58809170-p1m
El alineamiento se está realizando ahora mismo ...
El alineamiento se está realizando ahora mismo ...
El alineamiento se está realizando ahora mismo ...
El alineamiento se está realizando ahora mismo ...
El alineamiento se está realizando ahora mismo ...
El alineamiento se está realizando ahora mismo ...
El alineamiento se está realizando ahora mismo ...
El alineamiento se está realizando ahora mismo ...
CLUSTAL O(1.2.4) multiple sequence alignment


rs55863639        -AGGGGGAACGGC----AGGGTTCGTTACCTACTAAACTAC-------------------	36
rs786201057       -AGGGGGAACGGC----AGGGTTCGTTACCTACTAAACTAC-------------------	36
rs2073448940      -AGGGGGAACGGC----AGGGTTCGTTACCTACTAAACTAC-------------------	36
rs1555526478      -AGGGGGAACGGC----AGGGTTCGTTACCTACTAAACTAC-------------------	36
rs1267047192      -AGGGGGAACGGC----AGGGTTCGTTACCTACTAAACTAC-------------------	36
rs1567555504      -AGGGGGAACGGC----AGGGTTCGTTACCTACTAAACTAC-----

In [28]:
def descargar_alineamiento (resultado_alineamiento, nombre_archivo):
    with open(nombre_archivo, "w") as archivo:
        archivo.write(resultado_alineamiento)
    print(f"El alineamiento múltiple de las secuencias se ha guardado correctamente")

In [30]:
descargar_alineamiento (alineamiento_var_patogenicas_exon4_tp53, "data_TP53/alineamiento_var_patogenicas_exon4_tp53.aln-clustal_num")

El alineamiento múltiple de las secuencias se ha guardado correctamente
